# ニューストピック分類 学習



## Google ドライブの接続  
以下を実行。表示されるurlからGoogleアカウントにサイン。表示される認証用のコードを以下に表示されるテキストボックスにコピペ。  
→ サインインしたGoogleアカウントのGoogleドライブがマウントされる。

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


## 主な設定

In [2]:
data_path = '/content/drive/My Drive/Colab Notebooks/data'   # 作業用フォルダ

## ライブラリのインストール

In [3]:
!pip install transformers==4.10
!pip install datasets==1.11
!pip install fugashi==1.1
!pip install ipadic==1.0

     |████████████████████████████████| 2.8 MB 7.6 MB/s 
     |████████████████████████████████| 3.3 MB 58.8 MB/s 
     |████████████████████████████████| 636 kB 53.1 MB/s 
     |████████████████████████████████| 895 kB 63.5 MB/s 
     |████████████████████████████████| 50 kB 8.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 8.3 MB/s 
     |████████████████████████████████| 243 kB 63.5 MB/s 
     |████████████████████████████████| 119 kB 71.4 MB/s 
     |████████████████████████████████| 486 kB 8.0 MB/s 
     |████████████████████████████████| 13.4 MB 6.2 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=fca0a061951d01f3bbc5daf9c3df10b49cf42b5c84b99fe45d1ba7904be17efa
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully bui

## ModelとTokenizerの読み込み
日本語の事前学習済みモデルと、紐づいたTokenizerを読み込む。  
[cl-tohoku/bert-base-japanese-whole-word-masking](https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking)を使用する。

In [4]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
sc_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=9)
sc_model.cuda()
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

## データセットの読み込み
see. [huggingface datasets.load_dataset method](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset)

In [5]:
import os
from datasets import load_dataset
import pandas as pd
news_train = pd.read_csv(
    os.path.join(data_path, "news_train.csv"), index_col=False)
news_test = pd.read_csv(
    os.path.join(data_path, "news_test.csv" ), index_col=False)

In [6]:
news_train.head()

,text,label
0,「クリスマスだから、どこかリゾートに行きたい」と思いつつも、師走の忙しい最中に時間を作るのは...,5
1,節電モードの冬。各地のクリスマスイルミネーションも縮小気味らしい。そんな冬だからこそ、今年は...,0
2,世界屈指のクリエイターたちが丁寧に紡いだ『ももへの手紙』は、豊かな自然とやさしい人々が生きる...,5
3,いよいよ本格展開の「IPv6」がスマートフォンに投げかける課題とは？ 6/6から「World...,3
4,2002年の誕生から、今回で15回目となる「神戸コレクション2010 SPRING/SUMM...,5


In [7]:
news_test.head()

,text,label
0,2008年、”鉄の女”のニックネームを持つ元英国首相マーガレット・サッチャーが、実は認知症に...,5
1,箱根駅伝に出場し、4区を走った東京農業大学・内藤寛人選手の部屋の様子が日本テレビ系情報番組「...,4
2,キヤノンが女子に大人気の小物ブランド「Samantha Thavasa Petit Choi...,5
3,アレクサンドル・デュマの名作文学『三銃士』が、現代の壮大なエンタテインメント作品として生まれ...,5
4,悪質なコンピュータウイルスが氾濫しているWindowsに対して「Macは安全」とされる。Ma...,7


## 語数の評価

In [8]:
# 形態素解析
import torch
def tokenize(text):
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # インデックスに変換
    word_tensor = torch.tensor([word_ids])  # テンソルに変換
    return word_tensor

In [9]:
def count_words(texts):
    words_counts = []
    for text in texts:
        x = tokenize(text)
        words_counts.append(len(x[0]))
    return pd.DataFrame({'#_of_words':words_counts})

In [10]:
count_words(news_train['text']).describe()

,#_of_words
count,5525.000000
mean,723.745882
std,442.913901
min,4.000000
25%,417.000000
50%,605.000000
75%,941.000000
max,6501.000000


In [11]:
count_words(news_test['text']).describe()

,#_of_words
count,1842.000000
mean,715.496743
std,427.888321
min,16.000000
25%,409.000000
50%,599.000000
75%,934.000000
max,3674.000000


## データローダーのセット

In [12]:
train_data = load_dataset("csv", 
    data_files=os.path.join(data_path, "news_train.csv"), split="train")
print(train_data)
test_data  = load_dataset("csv", 
    data_files=os.path.join(data_path, "news_test.csv") , split="train")
print(test_data)

Using custom data configuration default-a7c6edb3c3579c48


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a7c6edb3c3579c48/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.
Dataset({
    features: ['text', 'label'],
    num_rows: 5525
})


Using custom data configuration default-92ce55445dde5da8


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-92ce55445dde5da8/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.
Dataset({
    features: ['text', 'label'],
    num_rows: 1842
})


## 形態素解析

In [13]:
MAX_LENGTH = 512
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, 
                     truncation=True, max_length=MAX_LENGTH)

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])

test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## 評価関数の定義
`sklearn.metrics`を使用。  


In [14]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

## Trainerの設定
see. [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#id1)  
see. [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments)  

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 2,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,  # 学習係数が0からこのステップ数で上昇
    weight_decay = 0.01,  # 重みの減衰率
    evaluation_strategy = "epoch",
    logging_dir = "./logs",
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data,
)

## モデルの訓練（fine tuning）


In [16]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 5525
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1382


Epoch,Training Loss,Validation Loss,Accuracy
1,0.976100,0.300549,0.914767
2,0.259000,0.232463,0.944083


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1842
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1842
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1382, training_loss=0.49288373745643627, metrics={'train_runtime': 1434.4955, 'train_samples_per_second': 7.703, 'train_steps_per_second': 0.963, 'total_flos': 2907559890892800.0, 'train_loss': 0.49288373745643627, 'epoch': 2.0})

## モデルの評価

In [17]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1842
  Batch size = 32


{'epoch': 2.0,
 'eval_accuracy': 0.9440825190010858,
 'eval_loss': 0.23246273398399353,
 'eval_runtime': 79.9956,
 'eval_samples_per_second': 23.026,
 'eval_steps_per_second': 0.725}

## モデルの保存

In [18]:
sc_model.save_pretrained(data_path)
tokenizer.save_pretrained(data_path)

Configuration saved in /content/drive/My Drive/Colab Notebooks/data/config.json
Model weights saved in /content/drive/My Drive/Colab Notebooks/data/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/Colab Notebooks/data/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/Colab Notebooks/data/special_tokens_map.json


('/content/drive/My Drive/Colab Notebooks/data/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/data/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/data/vocab.txt',
 '/content/drive/My Drive/Colab Notebooks/data/added_tokens.json')

## モデルの読み込み確認


In [19]:
loaded_model = BertForSequenceClassification.from_pretrained(data_path) 
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(data_path)

loading configuration file /content/drive/My Drive/Colab Notebooks/data/config.json
Model config BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 1